In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Pará - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_PA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - Desemprego,Pará - Produção de Cimento (t),Pará - value,Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Pará - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.348543,49.618560,0.312867,0.669564,7.776091e+07,4.250430e+06,9.220374,6.851269e+07,84.701
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.342554,49.658098,0.312657,0.669764,7.787181e+07,4.255476e+06,9.225493,6.856365e+07,80.221
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.336566,49.906876,0.311736,0.669963,7.798271e+07,4.260521e+06,9.230612,6.861461e+07,76.122
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.330578,49.864305,0.310493,0.670163,7.809361e+07,4.265566e+06,9.235731,6.866557e+07,76.544
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.324589,50.179753,0.309128,0.670362,7.820451e+07,4.270611e+06,9.240851,6.871652e+07,83.445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.899018,0.710173,NaN,NaN,NaN,NaN,NaN,156.442
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.416567,0.708384,NaN,NaN,NaN,NaN,NaN,145.777
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.559167,0.706038,NaN,NaN,NaN,NaN,NaN,139.233
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.696005,0.702147,NaN,NaN,NaN,NaN,NaN,142.025


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - Desemprego,Pará - Produção de Cimento (t),Pará - value,Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.743008,-1.860097,-1.001954,-1.906833,-1.652592,-1.810463,-2.287151,-1.950096
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.748918,-1.858558,-1.004769,-1.880912,-1.635387,-1.758101,-2.245800,-1.920309
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.754829,-1.848873,-1.017098,-1.854991,-1.618182,-1.705740,-2.204450,-1.890522
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.760740,-1.850531,-1.033739,-1.829071,-1.600977,-1.653378,-2.163099,-1.860734
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.766650,-1.838250,-1.052007,-1.803150,-1.583772,-1.601017,-2.121748,-1.830947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.076246,-0.644020,1.421704,1.389768,1.316401,-0.980846,1.292832,1.315283
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.073514,-0.676398,1.447588,1.378488,1.314484,-0.973932,1.302757,1.318820
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.070782,-0.703221,1.485640,1.367208,1.312566,-0.967018,1.312682,1.322356
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.068050,-0.725198,1.528780,1.355927,1.310648,-0.960105,1.322607,1.325893


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      110.204
1       76.354
2       77.171
3       80.720
4       82.088
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Pará - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - Desemprego,Pará - Produção de Cimento (t),Pará - value,Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.743008,-1.860097,-1.001954,-1.906833,-1.652592,-1.810463,-2.287151,-1.950096
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.748918,-1.858558,-1.004769,-1.880912,-1.635387,-1.758101,-2.245800,-1.920309
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.754829,-1.848873,-1.017098,-1.854991,-1.618182,-1.705740,-2.204450,-1.890522
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.760740,-1.850531,-1.033739,-1.829071,-1.600977,-1.653378,-2.163099,-1.860734
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.766650,-1.838250,-1.052007,-1.803150,-1.583772,-1.601017,-2.121748,-1.830947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.444243,0.582557,1.197817,1.368533,1.186547,-1.094003,0.861099,1.054711
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.421118,0.538378,1.208551,1.376125,1.197116,-1.099823,0.885880,1.071769
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.397993,0.482165,1.218732,1.383716,1.207686,-1.105643,0.910662,1.088827
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.374869,0.427013,1.224310,1.391308,1.218256,-1.111463,0.935444,1.105885


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      110.204
1       76.354
2       77.171
3       80.720
4       82.088
        ...   
157    116.872
158    131.959
159    116.171
160    132.242
161    135.615
Name: Pará - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - Desemprego,Pará - Produção de Cimento (t),Pará - value,Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
126,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.937142,1.114270,0.390386,0.491346,0.718639,0.747580,0.575976,0.685697
127,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.968970,1.082022,0.414807,0.506090,0.735578,0.698813,0.571078,0.692184
128,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,1.000798,1.052039,0.439827,0.520835,0.752516,0.650046,0.566181,0.698671
129,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,1.032625,1.020267,0.471477,0.535579,0.769455,0.601279,0.561283,0.705157
130,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,1.064453,0.982058,0.505972,0.550323,0.786394,0.552512,0.556385,0.711644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.076246,-0.644020,1.421704,1.389768,1.316401,-0.980846,1.292832,1.315283
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.073514,-0.676398,1.447588,1.378488,1.314484,-0.973932,1.302757,1.318820
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.070782,-0.703221,1.485640,1.367208,1.312566,-0.967018,1.312682,1.322356
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.068050,-0.725198,1.528780,1.355927,1.310648,-0.960105,1.322607,1.325893


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(36, activation='tanh', 
                                 return_sequences=True), 
                                 input_shape=(train_input.shape[1], 
                                              train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(180, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1198983599, 128333698, 2662745307, 519685290, 3233056670, 3950978624, 1214890807, 286635807, 2911121079, 2111195840, 4106323327, 2746996513, 96826094, 163209549, 518449701, 1916953013, 2910276772, 3833838257, 2496392057, 3612084860, 3569892794, 2718747398, 1547135396, 750241564, 1841362954, 2796892512, 1602773205, 717772582, 2705275597, 2185691279, 3963248111, 1968265083, 4292407657, 3194719252, 1226094749, 2149655597, 1602059599, 4185364796, 260342456, 3010989989, 2956950271, 3106131259, 1348095188, 1667031659, 3807524049, 2590149337, 1691508197, 42794948, 3143516604, 792967019]


Step: 0 ___________________________________________
val_loss: 107.92958068847656
winner_seed: 1198983599


Step: 1 ___________________________________________
val_loss: 129.907470703125


Step: 2 ___________________________________________
val_loss: 97.74781799316406
winner_seed: 2662745307


Step: 3 ___________________________________________
val_loss: 91.74954986572266
winner_seed: 519685290


Step: 4 ___

In [35]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 232ms/step - loss: 18812.4883 - val_loss: 20546.1367
Epoch 2/10000
4/4 [==============================] - 0s 20ms/step - loss: 17080.1934 - val_loss: 4167.9429
Epoch 3/10000
4/4 [==============================] - 0s 16ms/step - loss: 2471.1670 - val_loss: 2079.0737
Epoch 4/10000
4/4 [==============================] - 0s 17ms/step - loss: 1275.3688 - val_loss: 591.4828
Epoch 5/10000
4/4 [==============================] - 0s 18ms/step - loss: 1536.8671 - val_loss: 1351.1599
Epoch 6/10000
4/4 [==============================] - 0s 19ms/step - loss: 1187.2999 - val_loss: 3561.1167
Epoch 7/10000
4/4 [==============================] - 0s 18ms/step - loss: 3823.9744 - val_loss: 1271.4937
Epoch 8/10000
4/4 [==============================] - 0s 17ms/step - loss: 1058.5690 - val_loss: 983.3740
Epoch 9/10000
4/4 [==============================] - 0s 18ms/step - loss: 925.5969 - val_loss: 1139.6979
Epoch 10/10000
4/4 [=========================

Epoch 79/10000
4/4 [==============================] - 0s 20ms/step - loss: 211.6797 - val_loss: 195.5758
Epoch 80/10000
4/4 [==============================] - 0s 17ms/step - loss: 206.9511 - val_loss: 152.0917
Epoch 81/10000
4/4 [==============================] - 0s 19ms/step - loss: 192.3911 - val_loss: 353.9755
Epoch 82/10000
4/4 [==============================] - 0s 20ms/step - loss: 218.8056 - val_loss: 172.2984
Epoch 83/10000
4/4 [==============================] - 0s 18ms/step - loss: 206.2365 - val_loss: 245.5190
Epoch 84/10000
4/4 [==============================] - 0s 18ms/step - loss: 184.6364 - val_loss: 193.4347
Epoch 85/10000
4/4 [==============================] - 0s 17ms/step - loss: 188.8305 - val_loss: 176.1212
Epoch 86/10000
4/4 [==============================] - 0s 17ms/step - loss: 160.2732 - val_loss: 164.1905
Epoch 87/10000
4/4 [==============================] - 0s 19ms/step - loss: 183.5183 - val_loss: 167.6392
Epoch 88/10000
4/4 [==============================] - 0

4/4 [==============================] - 0s 18ms/step - loss: 118.1146 - val_loss: 150.7964
Epoch 157/10000
4/4 [==============================] - 0s 18ms/step - loss: 108.3948 - val_loss: 409.9050
Epoch 158/10000
4/4 [==============================] - 0s 18ms/step - loss: 242.0110 - val_loss: 342.7906
Epoch 159/10000
4/4 [==============================] - 0s 18ms/step - loss: 201.8977 - val_loss: 187.3413
Epoch 160/10000
4/4 [==============================] - 0s 19ms/step - loss: 159.1395 - val_loss: 201.9276
Epoch 161/10000
4/4 [==============================] - 0s 17ms/step - loss: 132.0174 - val_loss: 193.9383
Epoch 162/10000
4/4 [==============================] - 0s 17ms/step - loss: 156.3049 - val_loss: 150.8353
Epoch 163/10000
4/4 [==============================] - 0s 17ms/step - loss: 145.9603 - val_loss: 162.9488
Epoch 164/10000
4/4 [==============================] - 0s 17ms/step - loss: 133.8095 - val_loss: 182.2572
Epoch 165/10000
4/4 [==============================] - 0s 16ms

4/4 [==============================] - 0s 18ms/step - loss: 113.4842 - val_loss: 184.2570
Epoch 234/10000
4/4 [==============================] - 0s 16ms/step - loss: 102.4054 - val_loss: 200.9274
Epoch 235/10000
4/4 [==============================] - 0s 17ms/step - loss: 110.7652 - val_loss: 156.1105
Epoch 236/10000
4/4 [==============================] - 0s 18ms/step - loss: 87.6979 - val_loss: 191.7808
Epoch 237/10000
4/4 [==============================] - 0s 18ms/step - loss: 90.6325 - val_loss: 200.3608
Epoch 238/10000
4/4 [==============================] - 0s 19ms/step - loss: 100.7745 - val_loss: 187.7075
Epoch 239/10000
4/4 [==============================] - 0s 19ms/step - loss: 117.5615 - val_loss: 197.4738
Epoch 240/10000
4/4 [==============================] - 0s 18ms/step - loss: 97.2718 - val_loss: 303.5406
Epoch 241/10000
4/4 [==============================] - 0s 19ms/step - loss: 139.3581 - val_loss: 146.5345
Epoch 242/10000
4/4 [==============================] - 0s 17ms/st

Epoch 311/10000
4/4 [==============================] - 0s 18ms/step - loss: 71.9405 - val_loss: 161.2704
Epoch 312/10000
4/4 [==============================] - 0s 17ms/step - loss: 82.5013 - val_loss: 163.5811
Epoch 313/10000
4/4 [==============================] - 0s 17ms/step - loss: 68.2008 - val_loss: 165.8109
Epoch 314/10000
4/4 [==============================] - 0s 17ms/step - loss: 78.9981 - val_loss: 179.3838
Epoch 315/10000
4/4 [==============================] - 0s 17ms/step - loss: 89.8044 - val_loss: 187.8979
Epoch 316/10000
4/4 [==============================] - 0s 17ms/step - loss: 89.2966 - val_loss: 169.7784
Epoch 317/10000
4/4 [==============================] - 0s 17ms/step - loss: 71.6868 - val_loss: 176.1468
Epoch 318/10000
4/4 [==============================] - 0s 18ms/step - loss: 80.0669 - val_loss: 143.4225
Epoch 319/10000
4/4 [==============================] - 0s 17ms/step - loss: 74.1861 - val_loss: 184.4766
Epoch 320/10000
4/4 [==============================] - 

...

4/4 [==============================] - 0s 18ms/step - loss: 73.8689 - val_loss: 140.2570
Epoch 935/10000
4/4 [==============================] - 0s 17ms/step - loss: 78.0892 - val_loss: 158.2160
Epoch 936/10000
4/4 [==============================] - 0s 17ms/step - loss: 96.2941 - val_loss: 107.8498
Epoch 937/10000
4/4 [==============================] - 0s 16ms/step - loss: 67.2791 - val_loss: 160.6746
Epoch 938/10000
4/4 [==============================] - 0s 18ms/step - loss: 62.3847 - val_loss: 101.8578
Epoch 939/10000
4/4 [==============================] - 0s 18ms/step - loss: 75.8429 - val_loss: 103.2555
Epoch 940/10000
4/4 [==============================] - 0s 17ms/step - loss: 54.1347 - val_loss: 145.0762
Epoch 941/10000
4/4 [==============================] - 0s 17ms/step - loss: 76.8606 - val_loss: 157.3419
Epoch 942/10000
4/4 [==============================] - 0s 16ms/step - loss: 68.2956 - val_loss: 131.9684
Epoch 943/10000
4/4 [==============================] - 0s 16ms/step - l

4/4 [==============================] - 0s 18ms/step - loss: 48.9378 - val_loss: 140.7143
Epoch 1013/10000
4/4 [==============================] - 0s 17ms/step - loss: 67.6872 - val_loss: 115.9915
Epoch 1014/10000
4/4 [==============================] - 0s 16ms/step - loss: 37.8737 - val_loss: 114.4530
Epoch 1015/10000
4/4 [==============================] - 0s 17ms/step - loss: 58.9661 - val_loss: 103.6062
Epoch 1016/10000
4/4 [==============================] - 0s 17ms/step - loss: 48.5837 - val_loss: 105.4890
Epoch 1017/10000
4/4 [==============================] - 0s 16ms/step - loss: 44.0285 - val_loss: 96.1018
Epoch 1018/10000
4/4 [==============================] - 0s 17ms/step - loss: 38.7580 - val_loss: 124.4184
Epoch 1019/10000
4/4 [==============================] - 0s 17ms/step - loss: 43.2756 - val_loss: 146.6663
Epoch 1020/10000
4/4 [==============================] - 0s 17ms/step - loss: 46.3513 - val_loss: 94.0173
Epoch 1021/10000
4/4 [==============================] - 0s 17ms/s

4/4 [==============================] - 0s 16ms/step - loss: 38.3302 - val_loss: 100.8311
Epoch 1090/10000
4/4 [==============================] - 0s 18ms/step - loss: 43.3050 - val_loss: 133.4122
Epoch 1091/10000
4/4 [==============================] - 0s 16ms/step - loss: 53.1756 - val_loss: 101.1143
Epoch 1092/10000
4/4 [==============================] - 0s 18ms/step - loss: 35.5109 - val_loss: 106.2815
Epoch 1093/10000
4/4 [==============================] - 0s 16ms/step - loss: 37.1229 - val_loss: 90.8739
Epoch 1094/10000
4/4 [==============================] - 0s 16ms/step - loss: 28.5301 - val_loss: 98.3827
Epoch 1095/10000
4/4 [==============================] - 0s 17ms/step - loss: 27.7405 - val_loss: 90.4302
Epoch 1096/10000
4/4 [==============================] - 0s 18ms/step - loss: 37.5515 - val_loss: 99.1505
Epoch 1097/10000
4/4 [==============================] - 0s 17ms/step - loss: 38.4960 - val_loss: 133.6915
Epoch 1098/10000
4/4 [==============================] - 0s 16ms/ste

4/4 [==============================] - 0s 18ms/step - loss: 32.2514 - val_loss: 103.8288
Epoch 1167/10000
4/4 [==============================] - 0s 16ms/step - loss: 28.1097 - val_loss: 129.8145
Epoch 1168/10000
4/4 [==============================] - 0s 18ms/step - loss: 32.0163 - val_loss: 124.6097
Epoch 1169/10000
4/4 [==============================] - 0s 17ms/step - loss: 34.5094 - val_loss: 148.2363
Epoch 1170/10000
4/4 [==============================] - 0s 18ms/step - loss: 27.0032 - val_loss: 113.2583
Epoch 1171/10000
4/4 [==============================] - 0s 18ms/step - loss: 25.0918 - val_loss: 121.9588
Epoch 1172/10000
4/4 [==============================] - 0s 16ms/step - loss: 25.3364 - val_loss: 136.0583
Epoch 1173/10000
4/4 [==============================] - 0s 17ms/step - loss: 33.3648 - val_loss: 121.2934
Epoch 1174/10000
4/4 [==============================] - 0s 17ms/step - loss: 23.3900 - val_loss: 125.7486
Epoch 1175/10000
4/4 [==============================] - 0s 17ms

4/4 [==============================] - 0s 18ms/step - loss: 25.8357 - val_loss: 108.1836
Epoch 1244/10000
4/4 [==============================] - 0s 18ms/step - loss: 23.6790 - val_loss: 112.3532
Epoch 1245/10000
4/4 [==============================] - 0s 17ms/step - loss: 33.4727 - val_loss: 106.7407
Epoch 1246/10000
4/4 [==============================] - 0s 15ms/step - loss: 22.6014 - val_loss: 128.7051
Epoch 1247/10000
4/4 [==============================] - 0s 17ms/step - loss: 30.0831 - val_loss: 108.3432
Epoch 1248/10000
4/4 [==============================] - 0s 18ms/step - loss: 24.8361 - val_loss: 125.2729
Epoch 1249/10000
4/4 [==============================] - 0s 18ms/step - loss: 24.1627 - val_loss: 102.8159
Epoch 1250/10000
4/4 [==============================] - 0s 18ms/step - loss: 25.6694 - val_loss: 114.1028
Epoch 1251/10000
4/4 [==============================] - 0s 21ms/step - loss: 21.7541 - val_loss: 112.9955
Epoch 1251: early stopping


In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,125.514236,125.627739,123.644325,119.587837,116.414032,113.864204,112.901016,111.142159,110.197388,106.717232,101.684326,92.710487,87.383209,95.008682,115.167999,124.382591,122.73391,121.73484,123.894447,125.741676,123.953384,118.883423,111.752365,100.850029,90.191963,83.087502,78.724182,86.66925,86.604477,85.528557
Target,136.117,136.475,125.3,132.627,135.172,131.376,111.175,80.97,92.173,86.782,85.15,135.094,132.351,133.744,128.527,147.51,138.511,103.294,110.704,91.638,90.744,82.108,97.49,96.803,118.687,109.761,120.188,131.62,125.374,115.811
Error,10.602768,10.847267,1.655678,13.039162,18.757965,17.511803,1.726013,30.172157,18.024391,19.935234,16.534325,42.383507,44.967789,38.735321,13.358994,23.127403,15.777092,18.440842,13.190445,34.103676,33.209381,36.775421,14.262367,4.047028,28.495033,26.6735,41.463821,44.950745,38.769524,30.28244


In [38]:
display(mae)
display(mape)

23.39404

0.20928137

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[1360.0049]] - Target[1388.411]| =  Error: [[28.406128]]; MAPE:[[0.02045945]]
1/1 [==============================] - 0s 16ms/step
Ano-0: |Prediction[[1371.4866]] - Target[1353.424]| =  Error: [[18.062622]]; MAPE:[[0.01334587]]
1/1 [==============================] - 0s 22ms/step
Ano-5: |Prediction[[510.80597]] - Target[721.441]| =  Error: [[210.63501]]; MAPE:[[0.2919643]]


[array([[28.406128]], dtype=float32),
 array([[18.062622]], dtype=float32),
 array([[210.63501]], dtype=float32)]

85.701256

0.108589865